In [24]:
import xlrd
import xlwt
import numpy as np

In [25]:
stimuli_wb = xlrd.open_workbook('/Users/handwerkerd/Documents/experiments/FastLoc/fastloc_20191105/AllStimuliSets.xlsx') 
stimorder_wb = xlrd.open_workbook('/Users/handwerkerd/Documents/experiments/FastLoc/fastloc_20191105/fastloc.events.adv.1.xlsx')

AllVis = wb.sheet_by_name('VisProc') 
  
# For row 0 and column 0 
AllVis.cell_value(0, 5)

'bolt'

In [45]:
StimLabelSheetName=('VisProc', 'FalVisProc', 'AudProc', 'FalAudProc')
StimLabelOrder=('VisProc', 'FalVisProc', 'AudProc', 'AudProc')
NewConditionOrder=(1, 3, 2, 4)
for runidx in range(8):
    tmprun_wb = xlwt.Workbook()
    tmprun_ws = tmprun_wb.add_sheet('Run' + str(runidx+1))
    tmprun_ws.write(0, 0, 'Run')
    tmprun_ws.write(0, 1, 'TrialN')
    tmprun_ws.write(0, 2, 'Procedure')
    tmprun_ws.write(0, 3, 'Font')
    tmprun_ws.write(0, 4, 'StartTR2000')
    tmprun_ws.write(0, 5, 'ITI2000')  
    tmprun_ws.write(0, 6, 'StartTR650')
    tmprun_ws.write(0, 7, 'ITI650') 
    tmprun_ws.write(0, 8, 'Condition')
    tmprun_ws.write(0, 9, 'target1')
    tmprun_ws.write(0, 10, 'target2')
    tmprun_ws.write(0, 11, 'target3')
    tmprun_ws.write(0, 12, 'target4')
    tmprun_ws.write(1,0, runidx+1)
    tmprun_ws.write(1,1, 0)
    tmprun_ws.write(1,2, 'Null')
    tmprun_ws.write(1,3, 'Arial')
    tmprun_ws.write(1,4, 0)
    tmprun_ws.write(1,5, 12)
    tmprun_ws.write(1,6, 0)
    tmprun_ws.write(1,7, 0)
    tmprun_ws.write(1,8, 0)

    RandPerms = np.array([np.random.permutation(24), np.random.permutation(24), 
                          np.random.permutation(24), np.random.permutation(24)])
    RandPermsIdx = np.array([0, 0, 0, 0])
    tmporder = stimorder_wb.sheet_by_index(runidx)
    for stimidx in range(68):
        tmpcond = round(tmporder.cell_value(stimidx,0))
        #print(tmpcond)
        tmprun_ws.write(stimidx+2,0, runidx+1)
        tmprun_ws.write(stimidx+2,1, stimidx+1)
        tmprun_ws.write(stimidx+2,2, StimLabelOrder[tmpcond])
        if tmpcond==1:
            tmprun_ws.write(stimidx+2,3, 'Wingdings')
        else:
            tmprun_ws.write(stimidx+2,3, 'Arial')
        tmprun_ws.write(stimidx+2,4, round(tmporder.cell_value(stimidx,2)))
        tmprun_ws.write(stimidx+2,5, round(tmporder.cell_value(stimidx,4))+4)
        tmprun_ws.write(stimidx+2,6, 0)
        tmprun_ws.write(stimidx+2,7, 0)
        tmprun_ws.write(stimidx+2,8, NewConditionOrder[tmpcond])
        tmpstimtargets = stimuli_wb.sheet_by_name(StimLabelSheetName[tmpcond])
        tmprun_ws.write(stimidx+2,9, tmpstimtargets.cell_value(RandPerms[tmpcond,RandPermsIdx[tmpcond]], 2))
        tmprun_ws.write(stimidx+2,10, tmpstimtargets.cell_value(RandPerms[tmpcond,RandPermsIdx[tmpcond]], 3))
        tmprun_ws.write(stimidx+2,11, tmpstimtargets.cell_value(RandPerms[tmpcond,RandPermsIdx[tmpcond]], 4))
        tmprun_ws.write(stimidx+2,12, tmpstimtargets.cell_value(RandPerms[tmpcond,RandPermsIdx[tmpcond]], 5))
        RandPermsIdx[tmpcond] = RandPermsIdx[tmpcond]+1
    tmprun_ws.write(70,0, runidx+1)    
    tmprun_ws.write(70,1, 69)
    tmprun_ws.write(70,2, 'Null')
    tmprun_ws.write(70,3, 'Arial')
    tmprun_ws.write(70,4, round(tmporder.cell_value(67,2))+ round(tmporder.cell_value(67,4))+4)
    tmprun_ws.write(70,5, 12)
    tmprun_ws.write(70,6, 0)
    tmprun_ws.write(70,7, 0)
    tmprun_ws.write(70,8, 0)
    tmprun_wb.save('fastloc_newrun' + str(runidx+1) + '.xls')
    


In [16]:
RandPerms[0,:]

array([18, 11, 12,  7, 10,  3,  8, 17, 19,  2, 14,  6,  4,  1, 20, 21,  0,
       22, 15,  5,  9, 13, 16, 23])